In [1]:
# Necessary when connecting to a jupyterhub kernel running on daint via VScode. NOT required otherwise

import os

new_path = '/users/ajayaraj/scratch/tests/qtpyt-tests/AuBDA13CH2'
os.chdir(new_path)


In [2]:
import numpy as np
from matplotlib import pyplot as plt

import tqdm
from pathlib import Path
from ase.io import read

from qtpyt.tools import remove_pbc, expand_coupling
from qtpyt.lo import tools as lot
from qtpyt.surface import principallayer, tools, kpts
from qtpyt.basis import Basis
from qtpyt.base.greenfunction import GreenFunction
from qtpyt.base.selfenergy import SelfEnergy

import numpy as np
import sys
from pathlib import Path
from ase.io import read
from matplotlib import pyplot as plt

from scipy.interpolate import interp1d

import tqdm

from qtpyt.basis import Basis
from qtpyt.lo import tools as lot
from qtpyt.base.greenfunction import GreenFunction
from qtpyt.base.selfenergy import SelfEnergy
from qtpyt.tools import remove_pbc, expand_coupling
from qtpyt.surface import tools
from qtpyt.surface.principallayer import PrincipalSelfEnergy
from qtpyt.base import intgreenfunction
from gpaw import restart

OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


In [3]:
from qtpyt.surface.tools import prepare_leads_matrices

### Helper functions

In [4]:
def get_species_indices(atoms,species):
    indices = []
    for element in species:
        element_indices = atoms.symbols.search(element)
        indices.extend(element_indices)
    return sorted(indices)


### Control parameters

In [5]:
# cmap_name = 'custom_white_red'
# colors = [(1, 1, 1), (166/255, 4/255, 4/255)]
# n_bins = 100
# cm = LinearSegmentedColormap.from_list(cmap_name, colors, N=n_bins)
# norm = LogNorm(vmin=0.1, vmax=10)

In [6]:
E_ref, T_ref = np.load("published_reference/transmission/ET_dft.npy")

In [7]:
GPWDEVICEDIR = 'dft/device/'
BRIDGE_SPECIES = ("N", "C", "H")
GPWLEADSDIR = 'dft/leads/'

In [8]:
cc_path = Path(GPWDEVICEDIR)
pl_path = Path(GPWLEADSDIR)
gpwfile = f'{cc_path}/scatt.gpw'

atoms, calc = restart(gpwfile, txt=None)
fermi = calc.get_fermi_level()
nao_a = np.array([setup.nao for setup in calc.wfs.setups])
basis = Basis(atoms, nao_a)

H_lcao, S_lcao = np.load("dft/device/hs_cc_k.npy")


In [9]:
H_leads_lcao, S_leads_lcao = np.load(pl_path / 'hs_pl_k.npy')

basis_dict = {'Au': 9, 'H': 5, 'C': 13, 'N': 13}

leads_atoms = read(pl_path / 'leads.xyz')
leads_basis = Basis.from_dictionary(leads_atoms, basis_dict)

device_atoms = read(cc_path / 'scatt.xyz')
device_basis = Basis.from_dictionary(device_atoms, basis_dict)

# Define the number of repetitions (Nr) and unit cell repetition in the leads
Nr = (1, 5, 3)
unit_cell_rep_in_leads = (5, 5, 3)

nodes = [0,810,1116,1278,1584,2394]

# Define energy range and broadening factor for the Green's function calculation
de = 0.2
energies = np.arange(-3., 3. + de / 2., de).round(7)
eta = 1e-3


In [10]:
H_leads_lcao.shape

(38, 54, 54)

In [ ]:
# Prepare the k-points and matrices for the leads (Hamiltonian and overlap matrices) This is here because tests suggest that
# this step needs to be done prior to removing PBC in the aligning Hamiltonian

kpts_t, h_leads_kii, s_leads_kii, h_leads_kij, s_leads_kij = prepare_leads_matrices(
    H_leads_lcao, S_leads_lcao, unit_cell_rep_in_leads, align=(0, H_lcao[0, 0, 0]))

# Remove periodic boundary conditions (PBC) from the device Hamiltonian and overlap matrices
remove_pbc(device_basis, H_lcao)
remove_pbc(device_basis, S_lcao)

selfenergies = [None, None]
selfenergies[0] = principallayer.PrincipalSelfEnergy(kpts_t, (h_leads_kii, s_leads_kii), (h_leads_kij, s_leads_kij), Nr=Nr)
selfenergies[1] = principallayer.PrincipalSelfEnergy(kpts_t, (h_leads_kii, s_leads_kii), (h_leads_kij, s_leads_kij), Nr=Nr, id='right')


In [ ]:
# extract indices of the active space which include C and N atoms
active = np.where(np.isin(device_atoms.symbols,['C','N']))[0]

idx_active = device_basis[active].get_indices().reshape(active.size,13)
print(idx_active)
idx_lo_pz = idx_active[:,3].copy()  # what is this? it seems to be the indices of the pz orbitals in the los basis
iaopz = idx_active[:,2].copy()  # what is this?

# get embedding space as the difference
idx_embedding = np.setdiff1d(range(H_lcao.shape[-1]), idx_lo_pz)


In [ ]:
# get subdiagonalization matrix 
Us, eig = lot.subdiagonalize_atoms(device_basis, H_lcao[0], S_lcao[0], a=active)

# make sure that the projection onto the original pz AOs has the same sign for all pz LOs
# I am still not sure if this step is correct but now at least I understand the logic behind it
flip_sign = np.where(Us[iaopz,idx_lo_pz]<0.)[0]
for i in flip_sign:
    Us[np.ix_(idx_active[i],idx_active[i])] *= -1

#get subdiagonalized pair
H_subdiagonal = lot.rotate_matrix(H_lcao[0], Us)
S_subdiagonal = lot.rotate_matrix(S_lcao[0], Us)

In [ ]:
# make LO-pz subspace orthogonal to the rest of the scattering region
hs_active, hs_embedding, hs_ea_coupling, U = lot.extract_orthogonal_subspaces(H_subdiagonal, S_subdiagonal, idx_lo_pz) 

# expand_coupling(selfenergies[0], hs_embedding[0].shape[0])
# expand_coupling(selfenergies[1], hs_embedding[0].shape[0], id='right')

# get size of leads principal layer
nleads = np.prod(Nr)*len(leads_basis)

# get size of embedding space
nembedding = idx_embedding.size

# creates self-energy \Sigma_{A} describing the effect of the embedding on the active space
sigma = SelfEnergy(hs_embedding, hs_ea_coupling, selfenergies=
      [(np.ix_(range(nleads),range(nleads)), selfenergies[0]),
       (np.ix_(range(nembedding-nleads,nembedding),range(nembedding-nleads,nembedding)), selfenergies[1])])

In [ ]:
# If they are tuples, check the shape of each element within the tuple
for i, elem in enumerate(hs_embedding):
    print(f"Shape of hs_mm[{i}]:", elem.shape)

for i, elem in enumerate(hs_active):
    print(f"Shape of hs_ii[{i}]:", elem.shape)

In [ ]:
gf_active = GreenFunction(*hs_active, selfenergies=[(slice(None),sigma)])

In [ ]:
gf_active.selfenergies

In [ ]:
# Calculate the transmission function T for each energy in the defined range
T_active = np.empty(energies.size)
for e, energy in enumerate(energies):
    T_active[e] = gf_active.get_transmission(energy)  # Compute transmission at each energy point


### What do I need?

1. Compute T(E) from G_active using eqn.B3. This can be done by providing hs_active and sigma_a to a Greensfunction class. hs_active is obtained from extract_orthogonal_subspace.
2. I need to compute sigma_A using eqn. B8 which requires the hs_ae_coupling and Greens function of the embedding region. hs_ae_coupling is obtained from extract_orthogonal_subspace.
3. Embedding Greens function can be obtained by providing hs_embedding and embedded leads self energy to the Greens function class (eqn. B11). Embedded leads self energy needs to be computed using eqn. B12 which requires hs_le_coupling and Greens function for each lead.

Based on the above information, if I were to order the steps chronologically to compute various quantities, it would look like this

1. Get Green's functions for the left and right leads.
2. Get the H and S for the coupling between the leads and the embedding region.
3. Use the above matrices to compute the embedded leads self energy sigma_L,R for the left and right. This matrix would have dimensions of the embedding region.
4. Provide hs_embedding and sigma_L,R to Greensfunction class to compute embedding Green's function G_e.
5. Compute active self energy using G_e and hs_ae_coupling
6. Provide hs_active and sigma_a to Greensfunction class to compute active Green's function G_a
7. Compute T(E) using G_a.get_transmission.

In [ ]:
plt.figure(figsize=(8, 6))  # Set the figure size

# Plot the reference DFT data
plt.plot(E_ref, T_ref, label="references dft", color='blue')

# Plot the computed DFT data
plt.plot(energies, T_active, 'o', label="computed dft", color='red')

# Set the y-axis to log scale
plt.yscale("log")

# Set x-axis limits
plt.xlim(-2, 2)

# Set y-axis limits, bottom starting at 5e-5
plt.ylim(bottom=5e-5)

# Add a legend to distinguish between the reference and computed data
plt.legend()

# Add labels for the axes
plt.xlabel('Energy')
plt.ylabel('Transmission')

# Add a title (optional)
plt.title('Transmission vs Energy')

# Show the grid for better readability
plt.grid(True)

# Display the plot
plt.show()

In [ ]:
n_A = len(index_active_region)
sigma_active = np.empty((energies.size, n_A, n_A), dtype=complex)  # Create 3D array

# Initialize an array to store the trace for each energy
trace_sigma_active = np.empty(energies.size, dtype=complex)

# Loop through energies and compute the trace for each corresponding sigma_active
for e, energy in enumerate(energies):
    sigma_active[e, :, :] = hybr_active.retarded(energy)  # Store the n_A x n_A matrix in sigma_active[e]
    trace_sigma_active[e] = np.trace(sigma_active[e, :, :])  # Compute the trace and store it
    
# Now plot the trace vs energy
plt.figure(figsize=(8, 6))
plt.plot(energies, trace_sigma_active.real, label='Real part of Trace', color='b')
plt.plot(energies, trace_sigma_active.imag, label='Imaginary part of Trace', color='r', linestyle='--')

# Adding labels and title
plt.xlabel('Energy')
plt.ylabel('Trace of sigma_active')
plt.title('Trace of sigma_active vs Energy')
plt.legend()

# Show the plot
plt.grid(True)
plt.show()